# HuggingFace y Datasets

## Instrucciones de Instalacion

Antes de empezar, verificar si tienen instalado las librerias necesarias.
En su configuracion personal, pueden instalar desde el directorio base del repositorio usando `pip install -r installer/requirements.txt`,
ya sea para Windows o Linux.

En caso de usar Colab, pueden copiar el contenido en un archivo dentro de Colab, o en su propia nube.
Ingresar la linea de codigo: `!pip install -r <folder>/requirements.txt` dentro de una celda, y presionar en `Ejecutar celda`

### HuggingFace_Hub

Hugging Face Hub, el sitio web principal, es una plataforma central que permite a cualquiera descubrir, utilizar y contribuir con nuevos modelos y conjuntos de datos de última generación. Alberga una amplia variedad de modelos, con más de 10.000 disponibles públicamente.

In [ ]:
import soundfile as sf
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torch

CACHE_DIR="../models"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16

processor = AutoProcessor.from_pretrained(
    "facebook/musicgen-small",
    cache_dir=CACHE_DIR,
)
model = MusicgenForConditionalGeneration.from_pretrained(
    "facebook/musicgen-small",
    cache_dir=CACHE_DIR,
).to(device)
sampling_rate = model.config.audio_encoder.sampling_rate

caption = ""
extras = {}

inputs = processor(
    text=[caption],
    padding=True,
    return_tensors="pt",
    **extras
).to(device)

audio_out = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=1024).squeeze()
audio_out = audio_out.cpu().numpy()
name_out = "generated/music_out.wav"
sf.write(name_out, audio_out, sampling_rate)

Ahora con otro modelo:

In [ ]:
from diffusers import StableDiffusionPipeline
import numpy as np
import torch
import extras_libs
from matplotlib import pyplot as plt
from argparse import Namespace


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "riffusion/riffusion-model-v1",
    torch_dtype=torch.float16,
    cache_dir=CACHE_DIR,
)
pipe.unet = extras_libs.get_unet_traced(CACHE_DIR, device, dtype)
pipe = pipe.to(device)

# num_inference_steps = 50
alpha = 0.5
start = Namespace(
    # prompt="Okay, Up with it girl, rock with it girl, Swang with me",
    prompt="piano melodic pop",
    seed=42,
    denoising=0.75,
    guidance=7.0
)
end = Namespace(
    prompt="modified to be sang by vocals",
    seed=123,
    denoising=0.75,
    guidance=7.0
)

image = extras_libs.forward_riffuse_pipeline(pipe, device, dtype, start, end, alpha=alpha)
plt.imshow(image)



Convertimos de imagen a Sonido

In [ ]:
audio = extras_libs.image2audio(image, device)
Audio(audio, rate=44100)

### Ahora cargamos bark




In [ ]:
import torch
import soundfile as sf
from datetime import datetime
from transformers import AutoProcessor, BarkModel


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16

processor = AutoProcessor.from_pretrained(
    "suno/bark-small",
    cache_dir=CACHE_DIR,
)
model = BarkModel.from_pretrained(
    "suno/bark",
    cache_dir="./models",
).to(device)

model.enable_cpu_offload()
sampling_rate = model.generation_config.sample_rate

captions = ""
voice_preset = "v2/es_speaker_1"

inputs = processor(caption, voice_preset=voice_preset).to(device)
audio_out = model.generate(**inputs).squeeze().cpu().numpy()

time_now = datetime.now().strftime("%Y%m%d_%H%M%S")
name_out = f"generated/speech_{time_now}.wav"
sf.write(name_out, audio_out, sampling_rate)

Finalmente Descargamos T5-small y revisamos su salida

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-small")
model = T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-small")
input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Datasets

Datasets es una biblioteca para acceder y compartir fácilmente conjuntos de datos para tareas de audio, visión por computadora y procesamiento del lenguaje natural (NLP).

Cargue un conjunto (base) de datos en una sola línea de código y utilice los potentes métodos de procesamiento de datos para preparar rápidamente su conjunto de datos para entrenarlo en un modelo de aprendizaje profundo. Con el respaldo del formato Apache Arrow, procese grandes conjuntos de datos con lecturas sin copia y sin restricciones de memoria para lograr una velocidad y eficiencia óptimas.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ylacombe/google-chilean-spanish", cache_dir=CACHE_DIR)

Utilizando un conjunto de datos de texto, preprocesamos utilizando diferente funciones:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
dataset = dataset.map(lambda examples: tokenizer(examples["text"]), batched=True)

dataset[0]